Quora Insincere Question Classification

In [ ]:
import pandas as pd
import numpy as np

Loading the data

In [ ]:
train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')

In [ ]:
train.head()

Converting the target elements into array for ease of usage

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(train.target.values)

Spliting the train data into train and validation data

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train.question_text.values, y, stratify = y, random_state = 24, shuffle = True, test_size = 0.1)

Applying TF-IDF Vectorizer to vectorize the words in the documents for our machine to understand the data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
tfv.fit(list(x_train) + list(x_valid))
x_train_tfv = tfv.transform(x_train)
x_valid_tfv = tfv.transform(x_valid)

Using Logistic Regression to train the model and calculate the log loss

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1)
clf.fit(x_train_tfv, y_train)
predictions = clf.predict_proba(x_valid_tfv)
from sklearn.metrics import log_loss
print(log_loss(y_valid, predictions))

Now using Count Vectorizer instead of TF-IDF Vectorizer to compare the performance

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')
ctv.fit(list(x_train), list(x_valid))
x_train_ctv = ctv.transform(x_train)
x_valid_ctv = ctv.transform(x_valid)

In [ ]:
clf.fit(x_train_ctv, y_train)
predictions = clf.predict_proba(x_valid_ctv)
print(log_loss(y_valid, predictions))

Count Vectorizer works a little better compared to the other vectorizer

Now, trying some conventional methods. Starting with Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train_ctv, y_train)
predictions = mnb.predict_proba(x_valid_ctv)
print(log_loss(y_valid, predictions))